In [ ]:
#@title Import libraries we use for this notebook
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

import random
import pathlib
import shutil

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import helper functions we're going to use in this notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

# Car Models 3778 feature extraction



In [ ]:
# imagenet/efficientnet_v2_imagenet21k_ft1k_l/feature_vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

In [ ]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (512, 512)
BATCH_SIZE = 32

train_dir = '/kaggle/input/car-model-variants-and-images-dataset/train'
test_dir = '/kaggle/input/car-model-variants-and-images-dataset/test'

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Train images:")
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
)

print("Test images:")
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)


In [ ]:
class_names = list(test_data.class_indices.keys())
print("Class names: ", class_names)
print(f'Total classes: {len(class_names)}')

In [ ]:
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [ ]:
# Function to create a model from a URL
def create_model(model_url, num_classes):
  # Download the pretrained model and save it as a Keras layer
  feature_extraction_layer = hub.KerasLayer(model_url,
                                           trainable=False,
                                           name="feature_extraction_layer",
                                           input_shape=IMAGE_SHAPE+(3,))
   
  # Create our model
  model = tf.keras.Sequential([
      feature_extraction_layer,
      layers.Dense(num_classes, activation="softmax", name="output_layer")
  ])

  return model

In [ ]:
cars_resnet = create_model(resnet_url,
                     num_classes=train_data.num_classes)

cars_resnet.summary()

In [ ]:
# Compile our efficientnet model
cars_resnet.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                     optimizer=Adam(),
                     metrics=["accuracy"])

In [ ]:
# Fit our efficientnet model to the data
history_cars_resnet = cars_resnet.fit(train_data,
                                  epochs=5,
                                  steps_per_epoch=len(train_data),
                                  validation_data=test_data,
                                  validation_steps=len(test_data),
                                  callbacks=[create_tensorboard_callback(dir_name="tensorboard",
                                                                         experiment_name="car_models_3778_resnet_v1")])

In [ ]:
import joblib
joblib.dump(cars_resnet, '/kaggle/working/car_models_3778_resnet50v2_model_v1.pkl')